# Notes - This notebook is an extension of the discussion forund here: https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/discussion/484124

##### This aids to show th benefits of diversity of different modeling. In this notebook we see LB of 0.41, 0.37, and 0.36 yet when we average them we see far better scores of 0.31. However this does come with its warnings. This notebook uses a combonation of 2 stage and 1 stage which "could" be dramatically overfitting the LB depending on the outcome of the distribution of voters in the test set. I would personally suggest a deep review of the methods used in this notebook before you opt to use any strategies from it. Please reference the many warnings about this in the discussions for more details.

#### Lastly, please do check out the hard work of those who have created these datasets and notebooks in the first place! Or who combined them in @Majiaqi111 . A high LB score does not always mean a good solution!

# Model 1 - @andreasbis 0.41 
https://www.kaggle.com/code/andreasbis/hms-inference-lb-0-41

In [1]:
# Importing essential libraries
import gc
import os
import random
import warnings
import numpy as np
import pandas as pd
from IPython.display import display

# PyTorch for deep learning
import timm
import torch
import torch.nn as nn  
import torch.optim as optim
import torch.nn.functional as F

# torchvision for image processing and augmentation
import torchvision.transforms as transforms

# Suppressing minor warnings to keep the output clean
warnings.filterwarnings('ignore', category=Warning)

# Reclaim memory no longer in use.
gc.collect()

46

In [2]:
class Config:
    seed=42
    image_transform=transforms.Resize((512, 512))
    num_folds=5
    
# Set the seed for reproducibility across multiple libraries
def set_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    
set_seed(Config.seed)

In [3]:
# Load and store the trained models for each fold into a list
models = []

# Load ResNet34d
for i in range(Config.num_folds):
    # Create the same model architecture as during training
    model_resnet = timm.create_model('resnet34d', pretrained=False, num_classes=6, in_chans=1)
    
    # Load the trained weights from the corresponding file
    model_resnet.load_state_dict(torch.load(f'/kaggle/input/hms-train-resnet34d/resnet34d_fold{i}.pth', map_location=torch.device('cpu')))
    
    # Append the loaded model to the models list
    models.append(model_resnet)

# Reclaim memory no longer in use.
gc.collect()

# Load EfficientNetB0
for j in range(Config.num_folds):
    # Create the same model architecture as during training
    model_effnet_b0 = timm.create_model('efficientnet_b0', pretrained=False, num_classes=6, in_chans=1)
    
    # Load the trained weights from the corresponding file
    model_effnet_b0.load_state_dict(torch.load(f'/kaggle/input/hms-train-efficientnetb0/efficientnet_b0_fold{j}.pth', map_location=torch.device('cpu')))
    
    # Append the loaded model to the models list
    models.append(model_effnet_b0)
    
# Reclaim memory no longer in use.
gc.collect()
    
# Load EfficientNetB1
for k in range(Config.num_folds):
    # Create the same model architecture as during training
    model_effnet_b1 = timm.create_model('efficientnet_b1', pretrained=False, num_classes=6, in_chans=1)
    
    # Load the trained weights from the corresponding file
    model_effnet_b1.load_state_dict(torch.load(f'/kaggle/input/hms-train-efficientnetb1/efficientnet_b1_fold{k}.pth', map_location=torch.device('cpu')))
    
    # Append the loaded model to the models list
    models.append(model_effnet_b1)

# Reclaim memory no longer in use.
gc.collect()

18

In [4]:
# Load test data and sample submission dataframe
test_df = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/test.csv")
submission = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/sample_submission.csv")

# Merge the submission dataframe with the test data on EEG IDs
submission = submission.merge(test_df, on='eeg_id', how='left')

# Generate file paths for each spectrogram based on the EEG data in the submission dataframe
submission['path'] = submission['spectrogram_id'].apply(lambda x: f"/kaggle/input/hms-harmful-brain-activity-classification/test_spectrograms/{x}.parquet")

# Display the first few rows of the submission dataframe
display(submission.head())

# Reclaim memory no longer in use
gc.collect()

,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,spectrogram_id,patient_id,path
0,3911565283,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,853520,6885,/kaggle/input/hms-harmful-brain-activity-class...


0

In [5]:
# Define the weights for each model
weight_resnet34d = 0.26
weight_effnetb0 = 0.48
weight_effnetb1 = 0.26

# Get file paths for test spectrograms
paths = submission['path'].values
test_preds = []

# Generate predictions for each spectrogram using all models
for path in paths:
    eps = 1e-6
    # Read and preprocess spectrogram data
    data = pd.read_parquet(path)
    data = data.fillna(-1).values[:, 1:].T
    data = np.clip(data, np.exp(-6), np.exp(10))
    data = np.log(data)
    
    # Normalize the data
    data_mean = data.mean(axis=(0, 1))
    data_std = data.std(axis=(0, 1))
    data = (data - data_mean) / (data_std + eps)
    data_tensor = torch.unsqueeze(torch.Tensor(data), dim=0)
    data = Config.image_transform(data_tensor)

    test_pred = []
    
    # Generate predictions using all models
    for model in models:
        model.eval()
        with torch.no_grad():
            pred = F.softmax(model(data.unsqueeze(0)))[0]
            pred = pred.detach().cpu().numpy()
        test_pred.append(pred)
        
    # Combine predictions from all models using weighted voting
    weighted_pred = weight_resnet34d * np.mean(test_pred[:Config.num_folds], axis=0) + \
                     weight_effnetb0 * np.mean(test_pred[Config.num_folds:2*Config.num_folds], axis=0) + \
                     weight_effnetb1 * np.mean(test_pred[2*Config.num_folds:], axis=0)
    
    test_preds.append(weighted_pred)

# Convert the list of predictions to a NumPy array for further processing
test_preds = np.array(test_preds)

# Reclaim memory no longer in use
gc.collect()

0

In [6]:
# Load the sample submission file and update it with model predictions for each label
sub1 = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/sample_submission.csv")
labels = ['seizure', 'lpd', 'gpd', 'lrda', 'grda', 'other']

# Assign model predictions to respective columns in the submission DataFrame
for i in range(len(labels)):
    sub1[f'{labels[i]}_vote'] = test_preds[:, i]

display(sub1.head())

# Reclaim memory no longer in use.
gc.collect()

,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,3911565283,0.034403,0.103879,0.000818,0.371507,0.010815,0.478578


0

In [7]:
print(sub1.iloc[:,-6:].sum(axis=1).to_string())

0    1.0


# Model 2 - @minhsienweng 0.37
https://www.kaggle.com/code/minhsienweng/infer-features-head-starter

In [8]:
import os, random, sys
import tensorflow as tf
import tensorflow
import tensorflow.keras.backend as K
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import albumentations as albu
from scipy.signal import butter, lfilter
import librosa
from sklearn.model_selection import KFold, GroupKFold
import tensorflow.keras.backend as K, gc
from tensorflow.keras.layers import Input, Dense, Multiply, Add, Conv1D, Concatenate, LayerNormalization

In [9]:
LOAD_MODELS_FROM = '/kaggle/input/features-head-starter-models'
TARGETS = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']

In [10]:
# Seed the same seed to all 
def seed_everything(seed=42):
    np.random.seed(42)
    random.seed(42)
    tf.random.set_seed(42)

SEED = 42
seed_everything(SEED)

In [11]:
import ctypes
libc = ctypes.CDLL("libc.so.6")
def clear_memory():
    libc.malloc_trim(0)
    gc.collect()

In [12]:
# USE SINGLE GPU, MULTIPLE GPUS 
gpus = tf.config.list_physical_devices('GPU')
# WE USE MIXED PRECISION
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
if len(gpus)>1:
    strategy = tf.distribute.MirroredStrategy()
    print(f'Using {len(gpus)} GPUs')
else:
    strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    print(f'Using {len(gpus)} GPU')

Using 2 GPUs


In [13]:
from abc import ABC, abstractmethod 
  
FEATS2 = ['Fp1','T3','C3','O1','Fp2','C4','T4','O2']
FEAT2IDX = {x:y for x,y in zip(FEATS2,range(len(FEATS2)))}
FEATS = [['Fp1','F7','T3','T5','O1'],
         ['Fp1','F3','C3','P3','O1'],
         ['Fp2','F8','T4','T6','O2'],
         ['Fp2','F4','C4','P4','O2']]
USE_PROCESSED = True # Use processed downsampled Raw EEG 
    
class BaseDataGenerator():
    'Generates data for Keras'
    def __init__(self, data, specs, eeg_specs, raw_eegs, augment, mode, data_type): 
        self.data = data
        self.augment = augment
        self.mode = mode
        self.data_type = data_type
        self.specs = specs
        self.eeg_specs = eeg_specs
        self.raw_eegs = raw_eegs
        self.on_epoch_end()
        
    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, index):
        X, y = self.data_generation(index)
        if self.augment: X = self.augmentation(X)
        return X, y
    
    def __call__(self):
        for i in range(self.__len__()):
            yield self.__getitem__(i)
            
            if i == self.__len__()-1:
                self.on_epoch_end()
                
    def on_epoch_end(self):
        if self.mode=='train': 
            self.data = self.data.sample(frac=1).reset_index(drop=True)
    
    # Abstract method generate data based on the trained data type
    @abstractmethod
    def data_generation(self, index):
        pass
        
    def butter_lowpass_filter(self, data, cutoff_freq=20, sampling_rate=200, order=4):
        nyquist = 0.5 * sampling_rate
        normal_cutoff = cutoff_freq / nyquist
        b, a = butter(order, normal_cutoff, btype='low', analog=False)
        filtered_data = lfilter(b, a, data, axis=0)
        return filtered_data
    
    def resize(self, img,size):
        composition = albu.Compose([
                albu.Resize(size[0],size[1])
            ])
        return composition(image=img)['image']
            
    def augmentation(self, img):
        composition = albu.Compose([
                albu.HorizontalFlip(p=0.4)
            ])
        return composition(image=img)['image']

In [14]:
# Implementation class generates the data based on the data_type 
class DataGenerator(BaseDataGenerator):
    'Generates data for Keras'
    def __init__(self, data, specs, eeg_specs, raw_eegs, augment, 
                 mode, data_type): 
        super().__init__(data, specs, eeg_specs, raw_eegs, augment, mode, data_type)
    
    def data_generation(self, index):
        if self.data_type == 'KE':
            X,y = self.generate_all_specs(index)
        elif self.data_type == 'E' or self.data_type == 'K':
            X,y = self.generate_specs(index)
        elif self.data_type == 'R':
            X,y = self.generate_raw(index)
        elif self.data_type in ['ER','KR']:
            X1,y = self.generate_specs(index)
            X2,y = self.generate_raw(index)
            X = (X1,X2)
        elif self.data_type == 'KER':
            X1,y = self.generate_all_specs(index)
            X2,y = self.generate_raw(index)
            X = (X1,X2)
        return X,y
    
    def generate_all_specs(self, index):
        X = np.zeros((512,512,3),dtype='float32')
        y = np.zeros((6,),dtype='float32')
        
        row = self.data.iloc[index]
        if self.mode=='test': 
            offset = 0
        else:
            offset = int(row.offset/2)
        
        eeg = self.eeg_specs[row.eeg_id]
        spec = self.specs[row.spec_id]
        
        imgs = [spec[offset:offset+300,k*100:(k+1)*100].T for k in [0,2,1,3]] # to match kaggle with eeg
        img = np.stack(imgs,axis=-1)
        # LOG TRANSFORM SPECTROGRAM
        img = np.clip(img,np.exp(-4),np.exp(8))
        img = np.log(img)
            
        # STANDARDIZE PER IMAGE
        img = np.nan_to_num(img, nan=0.0)    
            
        mn = img.flatten().min()
        mx = img.flatten().max()
        ep = 1e-5
        img = 255 * (img - mn) / (mx - mn + ep)
        
        X[0_0+56:100+56,:256,0] = img[:,22:-22,0] # LL_k
        X[100+56:200+56,:256,0] = img[:,22:-22,2] # RL_k
        X[0_0+56:100+56,:256,1] = img[:,22:-22,1] # LP_k
        X[100+56:200+56,:256,1] = img[:,22:-22,3] # RP_k
        X[0_0+56:100+56,:256,2] = img[:,22:-22,2] # RL_k
        X[100+56:200+56,:256,2] = img[:,22:-22,1] # LP_k
        
        X[0_0+56:100+56,256:,0] = img[:,22:-22,0] # LL_k
        X[100+56:200+56,256:,0] = img[:,22:-22,2] # RL_k
        X[0_0+56:100+56,256:,1] = img[:,22:-22,1] # LP_k
        X[100+56:200+56,256:,1] = img[:,22:-22,3] # RP_K
        
        # EEG
        img = eeg
        mn = img.flatten().min()
        mx = img.flatten().max()
        ep = 1e-5
        img = 255 * (img - mn) / (mx - mn + ep)
        X[200+56:300+56,:256,0] = img[:,22:-22,0] # LL_e
        X[300+56:400+56,:256,0] = img[:,22:-22,2] # RL_e
        X[200+56:300+56,:256,1] = img[:,22:-22,1] # LP_e
        X[300+56:400+56,:256,1] = img[:,22:-22,3] # RP_e
        X[200+56:300+56,:256,2] = img[:,22:-22,2] # RL_e
        X[300+56:400+56,:256,2] = img[:,22:-22,1] # LP_e
        
        X[200+56:300+56,256:,0] = img[:,22:-22,0] # LL_e
        X[300+56:400+56,256:,0] = img[:,22:-22,2] # RL_e
        X[200+56:300+56,256:,1] = img[:,22:-22,1] # LP_e
        X[300+56:400+56,256:,1] = img[:,22:-22,3] # RP_e

        if self.mode!='test':
            y[:] = row[TARGETS]
        
        return X,y
    
    def generate_specs(self, index):
        X = np.zeros((512,512,3),dtype='float32')
        y = np.zeros((6,),dtype='float32')
        
        row = self.data.iloc[index]
        if self.mode=='test': 
            offset = 0
        else:
            offset = int(row.offset/2)
            
        if self.data_type in ['E','ER']:
            img = self.eeg_specs[row.eeg_id]
        elif self.data_type in ['K','KR']:
            spec = self.specs[row.spec_id]
            imgs = [spec[offset:offset+300,k*100:(k+1)*100].T for k in [0,2,1,3]] # to match kaggle with eeg
            img = np.stack(imgs,axis=-1)
            # LOG TRANSFORM SPECTROGRAM
            img = np.clip(img,np.exp(-4),np.exp(8))
            img = np.log(img)
            
            # STANDARDIZE PER IMAGE
            img = np.nan_to_num(img, nan=0.0)    
            
        mn = img.flatten().min()
        mx = img.flatten().max()
        ep = 1e-5
        img = 255 * (img - mn) / (mx - mn + ep)
        
        X[0_0+56:100+56,:256,0] = img[:,22:-22,0]
        X[100+56:200+56,:256,0] = img[:,22:-22,2]
        X[0_0+56:100+56,:256,1] = img[:,22:-22,1]
        X[100+56:200+56,:256,1] = img[:,22:-22,3]
        X[0_0+56:100+56,:256,2] = img[:,22:-22,2]
        X[100+56:200+56,:256,2] = img[:,22:-22,1]
        
        X[0_0+56:100+56,256:,0] = img[:,22:-22,0]
        X[100+56:200+56,256:,0] = img[:,22:-22,1]
        X[0_0+56:100+56,256:,1] = img[:,22:-22,2]
        X[100+56:200+56,256:,1] = img[:,22:-22,3]
        
        X[200+56:300+56,:256,0] = img[:,22:-22,0]
        X[300+56:400+56,:256,0] = img[:,22:-22,1]
        X[200+56:300+56,:256,1] = img[:,22:-22,2]
        X[300+56:400+56,:256,1] = img[:,22:-22,3]
        X[200+56:300+56,:256,2] = img[:,22:-22,3]
        X[300+56:400+56,:256,2] = img[:,22:-22,2]
        
        X[200+56:300+56,256:,0] = img[:,22:-22,0]
        X[300+56:400+56,256:,0] = img[:,22:-22,2]
        X[200+56:300+56,256:,1] = img[:,22:-22,1]
        X[300+56:400+56,256:,1] = img[:,22:-22,3]
        
        if self.mode!='test':
            y[:] = row[TARGETS]
        
        return X,y
    
    def generate_raw(self,index):
        if USE_PROCESSED and self.mode!='test':
            X = np.zeros((2_000,8),dtype='float32')
            y = np.zeros((6,),dtype='float32')
            row = self.data.iloc[index]
            X = self.raw_eegs[row.eeg_id]
            y[:] = row[TARGETS]
            return X,y
        
        X = np.zeros((10_000,8),dtype='float32')
        y = np.zeros((6,),dtype='float32')
        
        row = self.data.iloc[index]
        eeg = self.raw_eegs[row.eeg_id]
            
        # FEATURE ENGINEER
        X[:,0] = eeg[:,FEAT2IDX['Fp1']] - eeg[:,FEAT2IDX['T3']]
        X[:,1] = eeg[:,FEAT2IDX['T3']] - eeg[:,FEAT2IDX['O1']]
            
        X[:,2] = eeg[:,FEAT2IDX['Fp1']] - eeg[:,FEAT2IDX['C3']]
        X[:,3] = eeg[:,FEAT2IDX['C3']] - eeg[:,FEAT2IDX['O1']]
            
        X[:,4] = eeg[:,FEAT2IDX['Fp2']] - eeg[:,FEAT2IDX['C4']]
        X[:,5] = eeg[:,FEAT2IDX['C4']] - eeg[:,FEAT2IDX['O2']]
            
        X[:,6] = eeg[:,FEAT2IDX['Fp2']] - eeg[:,FEAT2IDX['T4']]
        X[:,7] = eeg[:,FEAT2IDX['T4']] - eeg[:,FEAT2IDX['O2']]
            
        # STANDARDIZE
        X = np.clip(X,-1024,1024)
        X = np.nan_to_num(X, nan=0) / 32.0
            
        # BUTTER LOW-PASS FILTER
        X = self.butter_lowpass_filter(X)
        # Downsample
        X = X[::5,:]
        
        if self.mode!='test':
            y[:] = row[TARGETS]
                
        return X,y

In [15]:
def create_dataset(data, mode, data_type, specs, eeg_specs, raw_eegs, augment=False,
                   batch_size=8):    
    if data_type in ['K','E','KE', 'K+E']: 
        inp = tf.TensorSpec(shape=(512,512,3), dtype=tf.float32)
    elif data_type in ['KR','ER','KER']:
        inp = (tf.TensorSpec(shape=(512,512,3), dtype=tf.float32), 
               tf.TensorSpec(shape=(2000,8), dtype=tf.float32))
    elif data_type in ['R']:
        inp = tf.TensorSpec(shape=(2000,8), dtype=tf.float32)

    output_signature = (inp, tf.TensorSpec(shape=(6,), dtype=tf.float32))
    
    
    # Create the data generator
    gen = DataGenerator(data, specs, eeg_specs, raw_eegs, augment, mode, data_type)
    # Create the dataset from data generator
    dataset = tf.data.Dataset.from_generator(generator=gen,
                                             output_signature=output_signature).batch(batch_size * strategy.num_replicas_in_sync)
    return dataset

In [16]:
def spectrogram_from_eeg(parquet_path):    
    # LOAD MIDDLE 50 SECONDS OF EEG SERIES
    eeg = pd.read_parquet(parquet_path)
    middle = (len(eeg)-10_000)//2
    eeg = eeg.iloc[middle:middle+10_000]
    
    # VARIABLE TO HOLD SPECTROGRAM
    img = np.zeros((100,300,4) ,dtype='float32')

    for k in range(4):
        COLS = FEATS[k]
        
        for kk in range(4):
            # FILL NANS
            x1 = eeg[COLS[kk]].values
            x2 = eeg[COLS[kk+1]].values
            m = np.nanmean(x1)
            if np.isnan(x1).mean()<1: 
                x1 = np.nan_to_num(x1,nan=m)
            else: 
                x1[:] = 0
            m = np.nanmean(x2)
            if np.isnan(x2).mean()<1: 
                x2 = np.nan_to_num(x2,nan=m)
            else: 
                x2[:] = 0
                
            # COMPUTE PAIR DIFFERENCES
            x = x1 - x2

            # RAW SPECTROGRAM
            mel_spec = librosa.feature.melspectrogram(y=x, sr=200, hop_length=len(x)//300, 
                                  n_fft=1024, n_mels=100, fmin=0, fmax=20, win_length=128)
            
            # LOG TRANSFORM
            width = (mel_spec.shape[1]//30)*30
            mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).astype(np.float32)[:,:width]
            img[:,:,k] += mel_spec_db
                
        # AVERAGE THE 4 MONTAGE DIFFERENCES
        img[:,:,k] /= 4.0
    return img
# Read EEG from par files
def eeg_from_parquet(parquet_path):
    eeg = pd.read_parquet(parquet_path, columns=FEATS2)
    rows = len(eeg)
    offset = (rows-10_000)//2
    eeg = eeg.iloc[offset:offset+10_000]
    data = np.zeros((10_000,len(FEATS2)))
    for j,col in enumerate(FEATS2):
        
        # FILL NAN
        x = eeg[col].values.astype('float32')
        m = np.nanmean(x)
        if np.isnan(x).mean()<1: x = np.nan_to_num(x,nan=m)
        else: x[:] = 0
        
        data[:,j] = x

    return data

In [17]:
def build_spec_model(hybrid=False):
    LOAD_BACKBONE_FROM = '/kaggle/input/efficientnetb-tf-keras/EfficientNetB2.h5'
    base_model = load_model(f'{LOAD_BACKBONE_FROM}')
    
    inp = tf.keras.layers.Input((512,512,3))        
    x = base_model(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    if not hybrid:
        x = tf.keras.layers.Dense(6,activation='softmax', dtype='float32')(x)
    # Create the model
    model_spec = tf.keras.Model(inputs=inp, outputs=x)
    opt = tf.keras.optimizers.Adam(learning_rate = 1e-3)
    loss = tf.keras.losses.KLDivergence()
    model_spec.compile(loss=loss, optimizer=opt)
    return model_spec

In [18]:
def wave_block(x, filters, kernel_size, n):
    dilation_rates = [2**i for i in range(n)]
    x = Conv1D(filters = filters,
               kernel_size = 1,
               padding = 'same')(x)
    res_x = x
    for dilation_rate in dilation_rates:
        tanh_out = Conv1D(filters = filters,
                          kernel_size = kernel_size,
                          padding = 'same', 
                          activation = 'tanh', 
                          dilation_rate = dilation_rate)(x)
        sigm_out = Conv1D(filters = filters,
                          kernel_size = kernel_size,
                          padding = 'same',
                          activation = 'sigmoid', 
                          dilation_rate = dilation_rate)(x)
        x = Multiply()([tanh_out, sigm_out])
        x = Conv1D(filters = filters,
                   kernel_size = 1,
                   padding = 'same')(x)
        res_x = Add()([res_x, x])
    return res_x
        
def build_wave_model(hybrid=False):
    # INPUT 
    inp = tf.keras.Input(shape=(2_000,8))

    ############
    # FEATURE EXTRACTION SUB MODEL
    inp2 = tf.keras.Input(shape=(2_000,1))
    x = wave_block(inp2, 8, 4, 6)
    x = wave_block(x, 16, 4, 6)
    x = wave_block(x, 32, 4, 6)
    x = wave_block(x, 64, 4, 6)
    base_model = tf.keras.Model(inputs=inp2, outputs=x)
    ###########

    # LEFT TEMPORAL CHAIN
    x1 = base_model(inp[:,:,0:1])
    x1 = tf.keras.layers.GlobalAveragePooling1D()(x1)
    x2 = base_model(inp[:,:,1:2])
    x2 = tf.keras.layers.GlobalAveragePooling1D()(x2)
    z1 = tf.keras.layers.Average()([x1,x2])

    # LEFT PARASAGITTAL CHAIN
    x1 = base_model(inp[:,:,2:3])
    x1 = tf.keras.layers.GlobalAveragePooling1D()(x1)
    x2 = base_model(inp[:,:,3:4])
    x2 = tf.keras.layers.GlobalAveragePooling1D()(x2)
    z2 = tf.keras.layers.Average()([x1,x2])

    # RIGHT PARASAGITTAL CHAIN
    x1 = base_model(inp[:,:,4:5])
    x1 = tf.keras.layers.GlobalAveragePooling1D()(x1)
    x2 = base_model(inp[:,:,5:6])
    x2 = tf.keras.layers.GlobalAveragePooling1D()(x2)
    z3 = tf.keras.layers.Average()([x1,x2])

    # RIGHT TEMPORAL CHAIN
    x1 = base_model(inp[:,:,6:7])
    x1 = tf.keras.layers.GlobalAveragePooling1D()(x1)
    x2 = base_model(inp[:,:,7:8])
    x2 = tf.keras.layers.GlobalAveragePooling1D()(x2)
    z4 = tf.keras.layers.Average()([x1,x2])

    # COMBINE CHAINS
    y = tf.keras.layers.Concatenate()([z1,z2,z3,z4])
    if not hybrid:
        y = tf.keras.layers.Dense(64, activation='relu')(y)
        y = tf.keras.layers.Dense(6,activation='softmax', dtype='float32')(y)

    # COMPILE MODEL
    model_wave = tf.keras.Model(inputs=inp, outputs=y)
    opt = tf.keras.optimizers.Adam(learning_rate = 1e-3)
    loss = tf.keras.losses.KLDivergence()
    model_wave.compile(loss=loss, optimizer = opt)
    return model_wave

In [19]:
def build_hyrbid_model():
    model_spec = build_spec_model(hybrid=True)
    model_wave = build_wave_model(hybrid=True)
    inputs = [model_spec.input, model_wave.input]
    x = [model_spec.output, model_wave.output]
    x = tf.keras.layers.Concatenate()(x)
    x = tf.keras.layers.Dense(6,activation='softmax', dtype='float32')(x)

    # COMPILE MODEL
    model_hybrid = tf.keras.Model(inputs=inputs, outputs=x)
    opt = tf.keras.optimizers.Adam(learning_rate = 1e-3)
    loss = tf.keras.losses.KLDivergence()
    model_hybrid.compile(loss=loss, optimizer = opt)
    return model_hybrid

In [20]:
model_spec = build_spec_model(hybrid=False)
model_wave = build_wave_model(hybrid=False)
model_hybrid = build_hyrbid_model()
# print(model_spec)

In [21]:
VERS ={
    'K': 43, # Kaggle spectrogram
    'E': 42, # EEG spectrogram 
    'R': 37, # Raw EEG 
    'KE': 47, # Kaggle + EEG spectrogram
    'KR': 48, # Kaggle spectrogram + Raw EEG 
    'ER': 49, # EEG spectrogram + Raw EEG  
    'KER': 50, # Kaggle + EGG spectrogram + Raw EEG 
    'K+E': 51, # Kaggle + EEG spectrogram + Data augumentation
}

In [22]:
from sklearn.model_selection import KFold, GroupKFold
import tensorflow.keras.backend as K, gc

# Load training data
def add_kl(data):
    labels = data[TARGETS].values + 1e-5 
    data['kl'] = tf.keras.losses.KLDivergence(reduction='none')(np.array([[1/6]*6]*len(data)), labels)
    return data
    
def load_train_data():
    # Load training features
    train = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
    META = ['spectrogram_id','spectrogram_label_offset_seconds','patient_id','expert_consensus']
    train = train.groupby('eeg_id')[META+TARGETS].agg({**{m:'first' for m in META},**{t:'sum' for t in TARGETS}}).reset_index() 
    train[TARGETS] = train[TARGETS]/train[TARGETS].values.sum(axis=1,keepdims=True)
    train.columns = ['eeg_id','spec_id','offset','patient_id','target'] + TARGETS
    train = add_kl(train)
    # display(train.head(3))
    # Read all spectrograms
    train_specs = np.load('/kaggle/input/brain-spectrograms/specs.npy',allow_pickle=True).item()
    # Read all EEG Spectrograms
    train_eegs = np.load('/kaggle/input/eeg-spectrograms/eeg_specs.npy',allow_pickle=True).item()
    # Read all raw EGG signals
    train_raw_eegs = np.load('/kaggle/input/hms-eeg/eegs_processed.npy',allow_pickle=True).item()
    return train, train_specs, train_eegs, train_raw_eegs

# Compute the score using KLDivergence 
# KL measures how much prediction prob distribution differs from actual prob distribution
def compute_score(y_true, y_pred):
    kl = tf.keras.metrics.KLDivergence()
    return kl(y_true, y_pred)

# Submission ON TEST with an individual
def preds_with_a_model(data_type):
    VER=VERS[data_type]
    train, train_specs, train_eegs, train_raw_eegs = load_train_data()
    # Make the predictions with 5 fold models
    all_oof = []
    all_true = []
    gkf = GroupKFold(n_splits=5)
    for i, (_, valid_index) in enumerate(gkf.split(train, train.target, train.patient_id)):   
        print(f'### Fold {i+1}')
        true_values = train.iloc[valid_index][TARGETS].values  
        all_true.append(true_values)
        # Valid dataset
        val_df = train.iloc[valid_index]
        val_dataset = create_dataset(val_df, mode='train', data_type=data_type,
                                     specs=train_specs, eeg_specs=train_eegs, 
                                     raw_eegs=train_raw_eegs)
        print(f'valid size {len(valid_index)}')
        model = None
        if data_type in ['K','E','KE']:
            model = model_spec
        elif data_type in ['R']:
            model = model_wave
        elif data_type in ['KR','ER','KER']:
            model = model_hybrid
        # EEG's, Kaggle's spectrograms and Raw model
        model.load_weights(f'{LOAD_MODELS_FROM}/model_{data_type}_{VER}_{i}.weights.h5')
        oof = model.predict(val_dataset, verbose=1)
        print(f"oof shape = {np.array(oof).shape}")
        all_oof.append(oof)           
        del val_df, val_dataset
        clear_memory()

    # Compute the score with predictions and actual labels
    all_oof = np.concatenate(all_oof)
    all_true = np.concatenate(all_true)
    print(f"all_oof shape = {all_oof.shape} and all_true shape = {all_true.shape}")
    print(f'CV KL SCORE of {data_type} model: {compute_score(all_true,all_oof)}')
    del train, train_specs, train_eegs, train_raw_eegs
    clear_memory()
    
# # Compute the cross validation score for a model
# preds_with_a_model(data_type='KE')
# sys.exit(0)

In [23]:
# Load testing features
test = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/test.csv')
# Rename
test = test.rename({'spectrogram_id':'spec_id'},axis=1)
print('Test shape',test.shape)
test.head()

Test shape (1, 3)


,spec_id,eeg_id,patient_id
0,853520,3911565283,6885


In [24]:
# Read all spectrograms
PATH = '/kaggle/input/hms-harmful-brain-activity-classification/test_spectrograms'
files = os.listdir(PATH)
print(f'There are {len(files)} test spectrogram parquets')
test_specs = {}
for i,f in enumerate(files):
    tmp = pd.read_parquet(f'{PATH}/{f}')
    name = int(f.split('.')[0])
    test_specs[name] = tmp.iloc[:,1:].values

There are 1 test spectrogram parquets


In [25]:
# Read all EEG Spectrograms
PATH = '/kaggle/input/hms-harmful-brain-activity-classification/test_eegs'
DISPLAY = 0
EEG_IDS = test.eeg_id.unique()
test_eeg_specs = {}
print('Converting Test EEG to Spectrograms...')
for i,eeg_id in enumerate(EEG_IDS):
    # CREATE SPECTROGRAM FROM EEG PARQUET
    test_eeg_specs[eeg_id] = spectrogram_from_eeg(f'{PATH}/{eeg_id}.parquet')

Converting Test EEG to Spectrograms...


In [26]:
# Read all RAW EEG Signals
test_raw_eegs = {}
EEG_IDS = test.eeg_id.unique()
PATH = '/kaggle/input/hms-harmful-brain-activity-classification/test_eegs'
print('Processing Test EEG parquets...')
for i,eeg_id in enumerate(EEG_IDS):
    # SAVE EEG TO PYTHON DICTIONARY OF NUMPY ARRAYS
    test_raw_eegs[eeg_id] = eeg_from_parquet(f'{PATH}/{eeg_id}.parquet')

Processing Test EEG parquets...


In [27]:
params = {'specs':test_specs, 'eeg_specs':test_eeg_specs, 'raw_eegs':test_raw_eegs}
test_datasets ={
    'K': create_dataset(test, data_type='K', mode='test', **params),
    'E': create_dataset(test, data_type='E', mode='test', **params),
    'R': create_dataset(test, data_type='R', mode='test', **params),
    'KE': create_dataset(test, data_type='KE', mode='test', **params),
    'KR': create_dataset(test, data_type='KR', mode='test', **params),
    'ER': create_dataset(test, data_type='ER', mode='test', **params),
    'KER': create_dataset(test, data_type='KER', mode='test', **params),
}

In [28]:
LBs = [0.41,0.39,0.41,0.37,0.39,0.38,0.36] # K|E|R|KE|KR|ER|KER for weighted ensemble we use LBs of each model

def make_preds(i, data_type):
    VER = VERS[data_type]
    test_dataset = test_datasets[data_type]
    model_preds = None
    if data_type in ['K','E','KE']:
        model_spec.load_weights(f'{LOAD_MODELS_FROM}/model_{data_type}_{VER}_{i}.weights.h5')
        model_preds = model_spec.predict(test_dataset, verbose=1)
    elif data_type in ['R']:
        model_wave.load_weights(f'{LOAD_MODELS_FROM}/model_{data_type}_{VER}_{i}.weights.h5')
        model_preds = model_wave.predict(test_dataset, verbose=1)
    elif data_type in ['KR','ER','KER']:
        model_hybrid.load_weights(f'{LOAD_MODELS_FROM}/model_{data_type}_{VER}_{i}.weights.h5')
        model_preds = model_hybrid.predict(test_dataset, verbose=1)
    clear_memory()
    return model_preds

# Submission ON TEST with ensemble
def preds_with_ensemble():
    preds = []
    # LB SCORE WEIGHTS FOR EACH MODEL
    lbs = 1 - np.array(LBs)
    weights = lbs/lbs.sum()
    for i in range(5):
        print(f'Fold {i+1}')
        # 'KE' model (Kaggle and EEG spectrogram)
        preds.append(make_preds(i, data_type='KE'))
        
        ## 'K' model (Kaggle spectrograms)
#         model_spec.load_weights(f'{LOAD_MODELS_FROM}/model_K_{VER_K}_{i}.weights.h5')
#         pred_K = model_spec.predict(test_dataset_K, verbose=1)
        ## EEG's spectrogram model
#         model_spec.load_weights(f'{LOAD_MODELS_FROM}/model_E_{VER_E}_{i}.weights.h5')
#         pred_E = model_spec.predict(test_dataset_E, verbose=1)
        # EEG Raw wavenet model
#         model_wave.load_weights(f'{LOAD_MODELS_FROM}/model_R_{VER_R}_{i}.weights.h5')
#         pred_R = model_wave.predict(test_dataset_R, verbose=1)
       
#         # Kaggle's spectrogram and Raw model
#         model_hybrid.load_weights(f'{LOAD_MODELS_FROM}/model_KR_{VER_KR}_{i}.weights.h5')
#         pred_KR = model_hybrid.predict(test_dataset_KR, verbose=1)
#         # EEG's spectrogram and Raw model
#         model_hybrid.load_weights(f'{LOAD_MODELS_FROM}/model_ER_{VER_ER}_{i}.weights.h5')
#         pred_ER = model_hybrid.predict(test_dataset_ER, verbose=1)
#         # EEG's, Kaggle's spectrograms and Raw model
#         model_hybrid.load_weights(f'{LOAD_MODELS_FROM}/model_KER_{VER_KER}_{i}.weights.h5')
#         pred_KER = model_hybrid.predict(test_dataset_KER, verbose=1)
        # Combine the predictions from all the model with different weights 
#         pred = np.array([pred_K,pred_E,pred_R,pred_KE,pred_KR,pred_ER,pred_KER])
#         pred = np.average(pred,axis=0,weights=weights)
        
#         preds.append(pred)
    # Average the prediction of all five fold models
    avg_pred = np.mean(preds, axis=0)
    clear_memory()
    return avg_pred
# Prediction with 
pred_final = preds_with_ensemble()
print('Test preds shape', pred_final.shape)

Fold 1
1/1 [==============================] - 5s 5s/step
Fold 2
1/1 [==============================] - 0s 139ms/step
Fold 3
1/1 [==============================] - 0s 156ms/step
Fold 4
1/1 [==============================] - 0s 144ms/step
Fold 5
1/1 [==============================] - 0s 150ms/step
Test preds shape (1, 6)


In [29]:
sub2 = pd.DataFrame({'eeg_id':test.eeg_id.values})
sub2[TARGETS] = pred_final
print('Submissionn shape',sub2.shape)
display(sub2.head())

Submissionn shape (1, 7)


,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,3911565283,0.0887,0.073478,0.000345,0.527459,0.011426,0.298592


In [30]:
print(sub2.iloc[:,-6:].sum(axis=1).to_string())

0    1.0


# Model 3 @konstantinboyko   0.36
https://www.kaggle.com/code/konstantinboyko/hms-full-validation-2-stage-s-train-infer/notebook?scriptVersionId=166580594

In [31]:
import os
import gc
import sys
import math
import time
import random
import datetime as dt
import numpy as np
import pandas as pd

from glob import glob
from pathlib import Path
from typing import Dict, List, Union
from scipy.signal import butter, lfilter, freqz
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

sys.path.append("/kaggle/input/kaggle-kl-div")
from kaggle_kl_div import score

import warnings

warnings.filterwarnings("ignore")

device = torch.device("cuda")
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

!cat /etc/os-release | grep -oP "PRETTY_NAME=\"\K([^\"]*)"
print(f"BUILD_DATE={os.environ['BUILD_DATE']}, CONTAINER_NAME={os.environ['CONTAINER_NAME']}")

try:
    print(
        f"PyTorch Version:{torch.__version__}, CUDA is available:{torch.cuda.is_available()}, Version CUDA:{torch.version.cuda}"
    )
    print(
        f"Device Capability:{torch.cuda.get_device_capability()}, {torch.cuda.get_arch_list()}"
    )
    print(
        f"CuDNN Enabled:{torch.backends.cudnn.enabled}, Version:{torch.backends.cudnn.version()}"
    )
except Exception:
    pass

Ubuntu 22.04.3 LTS
BUILD_DATE=20240109-221321, CONTAINER_NAME=tf2-gpu/2-13+gpu
PyTorch Version:2.0.0, CUDA is available:True, Version CUDA:11.8
Device Capability:(7, 5), ['sm_37', 'sm_60', 'sm_70', 'sm_75', 'compute_70', 'compute_75']
CuDNN Enabled:True, Version:8900


In [32]:
class APP:
    jupyter = "ipykernel" in globals()
    if not jupyter:
        try:
            if "IPython" in globals().get("__doc__", ""):
                jupyter = True
        except Exception as inst:
            print(inst)

    kaggle = os.environ.get("KAGGLE_KERNEL_RUN_TYPE", "") != ""
    local = os.environ.get("DOCKER_USING", "") == "LOCAL"
    date_time_start = dt.datetime.now()
    dt_start_ymd_hms = date_time_start.strftime("%Y.%m.%d_%H-%M-%S")

    file_run_path = ""
    if jupyter:
        try:
            file_run_path = Path(globals().get("__vsc_ipynb_file__", ""))
        except Exception as inst:
            print(inst)

    else:
        try:
            file_run_path = Path(__file__)
        except Exception as inst:
            print(inst)

    file_run_name = file_run_path.stem
    path_app = file_run_path.parent
    path_run = Path(os.getcwd())
    path_out = (
        Path("/kaggle/working")
        if kaggle
        else file_run_path / f"{file_run_name}_{dt_start_ymd_hms}"
    )


print(f"jupyter:{APP.jupyter}, kaggle:{APP.kaggle}, local:{APP.local}")
print(APP.file_run_path)
print(APP.path_out)

jupyter:True, kaggle:True, local:False
.
/kaggle/working


In [33]:
class CFG:
    VERSION = 93

    model_name = "resnet1d_gru"

    seed = 2024
    batch_size = 32
    num_workers = 0

    fixed_kernel_size = 5
    # kernels = [3, 5, 7, 9]
    # linear_layer_features = 424
    kernels = [3, 5, 7, 9, 11]
    #linear_layer_features = 448  # Full Signal = 10_000
    #linear_layer_features = 352  # Half Signal = 5_000
    linear_layer_features = 304   # 1/5  Signal = 2_000

    seq_length = 50  # Second's
    sampling_rate = 200  # Hz
    nsamples = seq_length * sampling_rate  # Число семплов
    out_samples = nsamples // 5

    # bandpass_filter = {"low": 0.5, "high": 20, "order": 2}
    # rand_filter = {"probab": 0.1, "low": 10, "high": 20, "band": 1.0, "order": 2}
    freq_channels = []  # [(8.0, 12.0)]; [(0.5, 4.5)]
    filter_order = 2
    random_close_zone = 0.0  # 0.2
        
    target_cols = [
        "seizure_vote",
        "lpd_vote",
        "gpd_vote",
        "lrda_vote",
        "grda_vote",
        "other_vote",
    ]

    # target_preds = [x + "_pred" for x in target_cols]
    # label_to_num = {"Seizure": 0, "LPD": 1, "GPD": 2, "LRDA": 3, "GRDA": 4, "Other": 5}
    # num_to_label = {v: k for k, v in label_to_num.items()}

    map_features = [
        ("Fp1", "T3"),
        ("T3", "O1"),
        ("Fp1", "C3"),
        ("C3", "O1"),
        ("Fp2", "C4"),
        ("C4", "O2"),
        ("Fp2", "T4"),
        ("T4", "O2"),
        #('Fz', 'Cz'), ('Cz', 'Pz'),        
    ]

    eeg_features = ["Fp1", "T3", "C3", "O1", "Fp2", "C4", "T4", "O2"]  # 'Fz', 'Cz', 'Pz']
        # 'F3', 'P3', 'F7', 'T5', 'Fz', 'Cz', 'Pz', 'F4', 'P4', 'F8', 'T6', 'EKG']                    
    feature_to_index = {x: y for x, y in zip(eeg_features, range(len(eeg_features)))}
    simple_features = []  # 'Fz', 'Cz', 'Pz', 'EKG'

    # eeg_features = [row for row in feature_to_index]
    # eeg_feat_size = len(eeg_features)
    
    n_map_features = len(map_features)
    in_channels = n_map_features + n_map_features * len(freq_channels) + len(simple_features)
    target_size = len(target_cols)
    
    PATH = "/kaggle/input/hms-harmful-brain-activity-classification/"
    test_eeg = "/kaggle/input/hms-harmful-brain-activity-classification/test_eegs/"
    test_csv = "/kaggle/input/hms-harmful-brain-activity-classification/test.csv"

In [34]:
koef_1 = 1.0
model_weights = [
    {
        'bandpass_filter':{'low':0.5, 'high':20, 'order':2}, 
        'file_data': 
        [
            #{'koef':koef_1, 'file_mask':"/kaggle/input/hms-resnet1d-gru-weights-v98/pop_1_weight_oof/*_full.pth"},
            #{'koef':koef_1, 'file_mask':"/kaggle/input/hms-resnet1d-gru-weights-v98/*_full.pth"},
            {'koef':koef_1, 'file_mask':"/kaggle/input/hms-resnet1d-gru-weights-v98/pop_2_weight_oof/*_full.pth"},
            #{'koef':koef_1, 'file_mask':"/kaggle/input/hms-resnet1d-gru-weights-v98/pop_3_weight_oof/*_full.pth"},
        ]
    },
]

In [35]:
def init_logger(log_file="./test.log"):
    from logging import getLogger, INFO, FileHandler, Formatter, StreamHandler

    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return "%dm %ds" % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return "%s (remain %s)" % (asMinutes(s), asMinutes(rs))


def quantize_data(data, classes):
    mu_x = mu_law_encoding(data, classes)
    return mu_x  # quantized


def mu_law_encoding(data, mu):
    mu_x = np.sign(data) * np.log(1 + mu * np.abs(data)) / np.log(mu + 1)
    return mu_x


def mu_law_expansion(data, mu):
    s = np.sign(data) * (np.exp(np.abs(data) * np.log(mu + 1)) - 1) / mu
    return s


def butter_bandpass(lowcut, highcut, fs, order=5):
    return butter(order, [lowcut, highcut], fs=fs, btype="band")


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y


def butter_lowpass_filter(
    data, cutoff_freq=20, sampling_rate=CFG.sampling_rate, order=4
):
    nyquist = 0.5 * sampling_rate
    normal_cutoff = cutoff_freq / nyquist
    b, a = butter(order, normal_cutoff, btype="low", analog=False)
    filtered_data = lfilter(b, a, data, axis=0)
    return filtered_data


def denoise_filter(x):
    # Частота дискретизации и желаемые частоты среза (в Гц).
    # Отфильтруйте шумный сигнал
    y = butter_bandpass_filter(x, CFG.lowcut, CFG.highcut, CFG.sampling_rate, order=6)
    y = (y + np.roll(y, -1) + np.roll(y, -2) + np.roll(y, -3)) / 4
    y = y[0:-1:4]
    return y

In [36]:
def eeg_from_parquet(
    parquet_path: str, display: bool = False, seq_length=CFG.seq_length
) -> np.ndarray:
    """
    Эта функция читает файл паркета и извлекает средние 50 секунд показаний. Затем он заполняет значения NaN
    со средним значением (игнорируя NaN).
        :param parquet_path: путь к файлу паркета.
        :param display: отображать графики ЭЭГ или нет.
        :return data: np.array формы (time_steps, eeg_features) -> (10_000, 8)
    """

    # Вырезаем среднюю 50 секундную часть
    eeg = pd.read_parquet(parquet_path, columns=CFG.eeg_features)
    rows = len(eeg)

    # начало смещения данных, чтобы забрать середину
    offset = (rows - CFG.nsamples) // 2

    # средние 50 секунд, имеет одинаковое количество показаний слева и справа
    eeg = eeg.iloc[offset : offset + CFG.nsamples]

    if display:
        plt.figure(figsize=(10, 5))
        offset = 0

    # Конвертировать в numpy

    # создать заполнитель той же формы с нулями
    data = np.zeros((CFG.nsamples, len(CFG.eeg_features)))

    for index, feature in enumerate(CFG.eeg_features):
        x = eeg[feature].values.astype("float32")  # конвертировать в float32

        # Вычисляет среднее арифметическое вдоль указанной оси, игнорируя NaN.
        mean = np.nanmean(x)
        nan_percentage = np.isnan(x).mean()  # percentage of NaN values in feature

        # Заполнение значения Nan
        # Поэлементная проверка на NaN и возврат результата в виде логического массива.
        if nan_percentage < 1:  # если некоторые значения равны Nan, но не все
            x = np.nan_to_num(x, nan=mean)
        else:  # если все значения — Nan
            x[:] = 0
        data[:, index] = x

        if display:
            if index != 0:
                offset += x.max()
            plt.plot(range(CFG.nsamples), x - offset, label=feature)
            offset -= x.min()

    if display:
        plt.legend()
        name = parquet_path.split("/")[-1].split(".")[0]
        plt.yticks([])
        plt.title(f"EEG {name}", size=16)
        plt.show()
    return data

In [37]:
class EEGDataset(Dataset):
    def __init__(
        self,
        df: pd.DataFrame,
        batch_size: int,
        eegs: Dict[int, np.ndarray],
        mode: str = "train",
        downsample: int = None,
        bandpass_filter: Dict[str, Union[int, float]] = None,
        rand_filter: Dict[str, Union[int, float]] = None,
    ):
        self.df = df
        self.batch_size = batch_size
        self.mode = mode
        self.eegs = eegs
        self.downsample = downsample
        self.bandpass_filter = bandpass_filter
        self.rand_filter = rand_filter
        
    def __len__(self):
        """
        Length of dataset.
        """
        # Обозначает количество пакетов за эпоху
        return len(self.df)

    def __getitem__(self, index):
        """
        Get one item.
        """
        # Сгенерировать один пакет данных
        X, y_prob = self.__data_generation(index)
        if self.downsample is not None:
            X = X[:: self.downsample, :]
        output = {
            "eeg": torch.tensor(X, dtype=torch.float32),
            "labels": torch.tensor(y_prob, dtype=torch.float32),
        }
        return output

    def __data_generation(self, index):
        # Генерирует данные, содержащие образцы размера партии
        X = np.zeros(
            (CFG.out_samples, CFG.in_channels), dtype="float32"
        )  # Size=(10000, 14)

        row = self.df.iloc[index]  # Строка Pandas
        data = self.eegs[row.eeg_id]  # Size=(10000, 8)
        if CFG.nsamples != CFG.out_samples:
            if self.mode != "train":
                offset = (CFG.nsamples - CFG.out_samples) // 2
            else:
                #offset = random.randint(0, CFG.nsamples - CFG.out_samples)                
                offset = ((CFG.nsamples - CFG.out_samples) * random.randint(0, 1000)) // 1000
            data = data[offset:offset+CFG.out_samples,:]

        for i, (feat_a, feat_b) in enumerate(CFG.map_features):
            if self.mode == "train" and CFG.random_close_zone > 0 and random.uniform(0.0, 1.0) <= CFG.random_close_zone:
                continue
                
            diff_feat = (
                data[:, CFG.feature_to_index[feat_a]]
                - data[:, CFG.feature_to_index[feat_b]]
            )  # Size=(10000,)

            if not self.bandpass_filter is None:
                diff_feat = butter_bandpass_filter(
                    diff_feat,
                    self.bandpass_filter["low"],
                    self.bandpass_filter["high"],
                    CFG.sampling_rate,
                    order=self.bandpass_filter["order"],
                )
                    
            if (
                self.mode == "train"
                and not self.rand_filter is None
                and random.uniform(0.0, 1.0) <= self.rand_filter["probab"]
            ):
                lowcut = random.randint(
                    self.rand_filter["low"], self.rand_filter["high"]
                )
                highcut = lowcut + self.rand_filter["band"]
                diff_feat = butter_bandpass_filter(
                    diff_feat,
                    lowcut,
                    highcut,
                    CFG.sampling_rate,
                    order=self.rand_filter["order"],
                )

            X[:, i] = diff_feat

        n = CFG.n_map_features
        if len(CFG.freq_channels) > 0:
            for i in range(CFG.n_map_features):
                diff_feat = X[:, i]
                for j, (lowcut, highcut) in enumerate(CFG.freq_channels):
                    band_feat = butter_bandpass_filter(
                        diff_feat, lowcut, highcut, CFG.sampling_rate, order=CFG.filter_order,  # 6
                    )
                    X[:, n] = band_feat
                    n += 1

        for spml_feat in CFG.simple_features:
            feat_val = data[:, CFG.feature_to_index[spml_feat]]
            
            if not self.bandpass_filter is None:
                feat_val = butter_bandpass_filter(
                    feat_val,
                    self.bandpass_filter["low"],
                    self.bandpass_filter["high"],
                    CFG.sampling_rate,
                    order=self.bandpass_filter["order"],
                )

            if (
                self.mode == "train"
                and not self.rand_filter is None
                and random.uniform(0.0, 1.0) <= self.rand_filter["probab"]
            ):
                lowcut = random.randint(
                    self.rand_filter["low"], self.rand_filter["high"]
                )
                highcut = lowcut + self.rand_filter["band"]
                feat_val = butter_bandpass_filter(
                    feat_val,
                    lowcut,
                    highcut,
                    CFG.sampling_rate,
                    order=self.rand_filter["order"],
                )

            X[:, n] = feat_val
            n += 1
            
        # Обрезать края превышающие значения [-1024, 1024]
        X = np.clip(X, -1024, 1024)

        # Замените NaN нулем и разделить все на 32
        X = np.nan_to_num(X, nan=0) / 32.0

        # обрезать полосовым фильтром верхнюю границу в 20 Hz.
        X = butter_lowpass_filter(X, order=CFG.filter_order)  # 4

        y_prob = np.zeros(CFG.target_size, dtype="float32")  # Size=(6,)
        if self.mode != "test":
            y_prob = row[CFG.target_cols].values.astype(np.float32)

        return X, y_prob

In [38]:
class ResNet_1D_Block(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size,
        stride,
        padding,
        downsampling,
        dilation=1,
        groups=1,
        dropout=0.0,
    ):
        super(ResNet_1D_Block, self).__init__()

        self.bn1 = nn.BatchNorm1d(num_features=in_channels)
        # self.relu = nn.ReLU(inplace=False)
        # self.relu_1 = nn.PReLU()
        # self.relu_2 = nn.PReLU()
        self.relu_1 = nn.Hardswish()
        self.relu_2 = nn.Hardswish()

        self.dropout = nn.Dropout(p=dropout, inplace=False)
        self.conv1 = nn.Conv1d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            dilation=dilation,
            groups=groups,
            bias=False,
        )

        self.bn2 = nn.BatchNorm1d(num_features=out_channels)
        self.conv2 = nn.Conv1d(
            in_channels=out_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            dilation=dilation,
            groups=groups,
            bias=False,
        )

        self.maxpool = nn.MaxPool1d(
            kernel_size=2,
            stride=2,
            padding=0,
            dilation=dilation,
        )
        self.downsampling = downsampling

    def forward(self, x):
        identity = x

        out = self.bn1(x)
        out = self.relu_1(out)
        out = self.dropout(out)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.relu_2(out)
        out = self.dropout(out)
        out = self.conv2(out)

        out = self.maxpool(out)
        identity = self.downsampling(x)

        out += identity
        return out


class EEGNet(nn.Module):
    def __init__(
        self,
        kernels,
        in_channels,
        fixed_kernel_size,
        num_classes,
        linear_layer_features,
        dilation=1,
        groups=1,
    ):
        super(EEGNet, self).__init__()
        self.kernels = kernels
        self.planes = 24
        self.parallel_conv = nn.ModuleList()
        self.in_channels = in_channels

        for i, kernel_size in enumerate(list(self.kernels)):
            sep_conv = nn.Conv1d(
                in_channels=in_channels,
                out_channels=self.planes,
                kernel_size=(kernel_size),
                stride=1,
                padding=0,
                dilation=dilation,
                groups=groups,
                bias=False,
            )
            self.parallel_conv.append(sep_conv)

        self.bn1 = nn.BatchNorm1d(num_features=self.planes)
        # self.relu = nn.ReLU(inplace=False)
        # self.relu_1 = nn.ReLU()
        # self.relu_2 = nn.ReLU()
        self.relu_1 = nn.SiLU()
        self.relu_2 = nn.SiLU()

        self.conv1 = nn.Conv1d(
            in_channels=self.planes,
            out_channels=self.planes,
            kernel_size=fixed_kernel_size,
            stride=2,
            padding=2,
            dilation=dilation,
            groups=groups,
            bias=False,
        )

        self.block = self._make_resnet_layer(
            kernel_size=fixed_kernel_size,
            stride=1,
            dilation=dilation,
            groups=groups,
            padding=fixed_kernel_size // 2,
        )
        self.bn2 = nn.BatchNorm1d(num_features=self.planes)
        self.avgpool = nn.AvgPool1d(kernel_size=6, stride=6, padding=2)

        self.rnn = nn.GRU(
            input_size=self.in_channels,
            hidden_size=128,
            num_layers=1,
            bidirectional=True,
            # dropout=0.2,
        )

        self.fc = nn.Linear(in_features=linear_layer_features, out_features=num_classes)

    def _make_resnet_layer(
        self,
        kernel_size,
        stride,
        dilation=1,
        groups=1,
        blocks=9,
        padding=0,
        dropout=0.0,
    ):
        layers = []
        downsample = None
        base_width = self.planes

        for i in range(blocks):
            downsampling = nn.Sequential(
                nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
            )
            layers.append(
                ResNet_1D_Block(
                    in_channels=self.planes,
                    out_channels=self.planes,
                    kernel_size=kernel_size,
                    stride=stride,
                    padding=padding,
                    downsampling=downsampling,
                    dilation=dilation,
                    groups=groups,
                    dropout=dropout,
                )
            )
        return nn.Sequential(*layers)

    def extract_features(self, x):
        x = x.permute(0, 2, 1)
        out_sep = []

        for i in range(len(self.kernels)):
            sep = self.parallel_conv[i](x)
            out_sep.append(sep)

        out = torch.cat(out_sep, dim=2)
        out = self.bn1(out)
        out = self.relu_1(out)
        out = self.conv1(out)

        out = self.block(out)
        out = self.bn2(out)
        out = self.relu_2(out)
        out = self.avgpool(out)

        out = out.reshape(out.shape[0], -1)
        rnn_out, _ = self.rnn(x.permute(0, 2, 1))
        new_rnn_h = rnn_out[:, -1, :]  # <~~

        new_out = torch.cat([out, new_rnn_h], dim=1)
        return new_out

    def forward(self, x):
        new_out = self.extract_features(x)
        result = self.fc(new_out)
        return result

In [39]:
def inference_function(test_loader, model, device):
    model.eval()  # set model in evaluation mode
    softmax = nn.Softmax(dim=1)
    prediction_dict = {}
    preds = []
    with tqdm(test_loader, unit="test_batch", desc="Inference") as tqdm_test_loader:
        for step, batch in enumerate(tqdm_test_loader):
            X = batch.pop("eeg").to(device)  # send inputs to `device`
            batch_size = X.size(0)
            with torch.no_grad():
                y_preds = model(X)  # forward propagation pass
            y_preds = softmax(y_preds)
            preds.append(y_preds.to("cpu").numpy())  # save predictions

    prediction_dict["predictions"] = np.concatenate(
        preds
    )  # np.array() of shape (fold_size, target_cols)
    return prediction_dict

In [40]:
test_df = pd.read_csv(CFG.test_csv)
print(f"Test dataframe shape is: {test_df.shape}")
test_df.head()

Test dataframe shape is: (1, 3)


,spectrogram_id,eeg_id,patient_id
0,853520,3911565283,6885


In [41]:
test_eeg_parquet_paths = glob(CFG.test_eeg + "*.parquet")
test_eeg_df = pd.read_parquet(test_eeg_parquet_paths[0])
test_eeg_features = test_eeg_df.columns
print(f"There are {len(test_eeg_features)} raw eeg features")
print(list(test_eeg_features))
del test_eeg_df
_ = gc.collect()

# %%time
all_eegs = {}
eeg_ids = test_df.eeg_id.unique()
for i, eeg_id in tqdm(enumerate(eeg_ids)):
    # Save EEG to Python dictionary of numpy arrays
    eeg_path = CFG.test_eeg + str(eeg_id) + ".parquet"
    data = eeg_from_parquet(eeg_path)
    all_eegs[eeg_id] = data

There are 20 raw eeg features
['Fp1', 'F3', 'C3', 'P3', 'F7', 'T3', 'T5', 'O1', 'Fz', 'Cz', 'Pz', 'Fp2', 'F4', 'C4', 'P4', 'F8', 'T4', 'T6', 'O2', 'EKG']


0it [00:00, ?it/s]

In [42]:
koef_sum = 0
koef_count = 0
predictions = []
files = []
    
for model_block in model_weights:
    test_dataset = EEGDataset(
        df=test_df,
        batch_size=CFG.batch_size,
        mode="test",
        eegs=all_eegs,
        bandpass_filter=model_block['bandpass_filter']
    )

    if len(predictions) == 0:
        output = test_dataset[0]
        X = output["eeg"]
        print(f"X shape: {X.shape}")
                
    test_loader = DataLoader(
        test_dataset,
        batch_size=CFG.batch_size,
        shuffle=False,
        num_workers=CFG.num_workers,
        pin_memory=True,
        drop_last=False,
    )

    model = EEGNet(
        kernels=CFG.kernels,
        in_channels=CFG.in_channels,
        fixed_kernel_size=CFG.fixed_kernel_size,
        num_classes=CFG.target_size,
        linear_layer_features=CFG.linear_layer_features,
    )

    for file_line in model_block['file_data']:
        koef = file_line['koef']
        for weight_model_file in glob(file_line['file_mask']):
            files.append(weight_model_file)
            checkpoint = torch.load(weight_model_file, map_location=device)
            model.load_state_dict(checkpoint["model"])
            model.to(device)
            prediction_dict = inference_function(test_loader, model, device)
            predict = prediction_dict["predictions"]
            predict *= koef
            koef_sum += koef
            koef_count += 1
            predictions.append(predict)
            torch.cuda.empty_cache()
            gc.collect()

predictions = np.array(predictions)
koef_sum /= koef_count
predictions /= koef_sum
predictions = np.mean(predictions, axis=0)

X shape: torch.Size([2000, 8])


Inference:   0%|          | 0/1 [00:00<?, ?test_batch/s]

Inference:   0%|          | 0/1 [00:00<?, ?test_batch/s]

Inference:   0%|          | 0/1 [00:00<?, ?test_batch/s]

Inference:   0%|          | 0/1 [00:00<?, ?test_batch/s]

Inference:   0%|          | 0/1 [00:00<?, ?test_batch/s]

In [43]:
print(koef_count, koef_sum)
display(files)

5 1.0


['/kaggle/input/hms-resnet1d-gru-weights-v98/pop_2_weight_oof/resnet1d_gru_ver-64_stage-2_fold-1_full.pth',
 '/kaggle/input/hms-resnet1d-gru-weights-v98/pop_2_weight_oof/resnet1d_gru_ver-64_stage-2_fold-4_full.pth',
 '/kaggle/input/hms-resnet1d-gru-weights-v98/pop_2_weight_oof/resnet1d_gru_ver-64_stage-2_fold-2_full.pth',
 '/kaggle/input/hms-resnet1d-gru-weights-v98/pop_2_weight_oof/resnet1d_gru_ver-64_stage-2_fold-0_full.pth',
 '/kaggle/input/hms-resnet1d-gru-weights-v98/pop_2_weight_oof/resnet1d_gru_ver-64_stage-2_fold-3_full.pth']

In [44]:
sub3 = pd.DataFrame({"eeg_id": test_df.eeg_id.values})
sub3[CFG.target_cols] = predictions

sub3.to_csv(f"submission.csv", index=False)
print(f"Submission shape: {sub3.shape}")
display(sub3.head())

Submission shape: (1, 7)


,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,3911565283,0.016862,0.026578,0.009426,0.093476,0.089352,0.764305


In [45]:
print(sub3.iloc[:,-6:].sum(axis=1).to_string())

0    1.0


# Submission

In [46]:
sub_final = pd.DataFrame({"eeg_id": test_df.eeg_id.values})
labels = ['seizure', 'lpd', 'gpd', 'lrda', 'grda', 'other']

for i in range(len(labels)):
    sub1[f'{labels[i]}_vote'] = test_preds[:, i]
sub2[TARGETS] = pred_final
sub3[CFG.target_cols] = predictions
    
for label in labels:
    sub_final[f'{label}_vote'] = (sub1[f'{label}_vote'] + sub2[f'{label}_vote'] + sub3[f'{label}_vote']) / 3.0 

sub_final.to_csv(f"submission.csv", index=False)
print(f"Submission shape: {sub_final.shape}")
display(sub_final.head())

Submission shape: (1, 7)


,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,3911565283,0.046655,0.067978,0.00353,0.330814,0.037198,0.513825


In [47]:
print(sub_final.iloc[:,-6:].sum(axis=1).to_string())

0    1.0
